In [1]:
import sys
#!conda remove lxml
#!conda install --yes lxml

In [45]:
import os
import re
import time
import glob
import numpy as np
import pandas as pd
import lxml
from requests import get
from bs4 import BeautifulSoup
from basketball_reference_scraper.teams import get_roster_stats as grs
from basketball_reference_scraper.utils import remove_accents
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
from constants import DATA_DIR #TEAMS, SECONDS_SLEEP

In [186]:
class DataScraper():
    def get_roster_stats1(self, team, )
    def get_roster_stats(self, team, season_end_year, data_format='PER_GAME', playoffs=False):
        if playoffs:
            period = 'playoffs'
        else:
            period = 'leagues'
        df = pd.DataFrame
        selector = data_format.lower()
        r = get(f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2F{period}%2FNBA_{season_end_year}_{selector}.html&div=div_{selector}_stats')
        df = None
        if r.status_code == 200:
            soup = BeautifulSoup(r.content, 'html.parser')
            table = soup.find('table')
            df2 = pd.read_html(str(table))[0]
            df2.to_csv('asdsfdf.csv')
            for index, row in df2.iterrows():
                if row['Tm'] == team:
                    if df is None:
                        df = pd.DataFrame(columns=list(row.index)+['SEASON'])
                    row['SEASON'] = f'{season_end_year-1}-{str(season_end_year)[2:]}'
                    df = df.append(row)
            df.rename(columns={'Player':'PLAYER','Age':'AGE','Tm':'TEAM','Pos': 'POS'}, inplace=True)
            df['PLAYER'] = df['PLAYER'].apply(lambda name: remove_accents(name, team, season_end_year))
            df = df.reset_index().drop(['Rk','index','TEAM','AGE'], axis=1)
        return df
    # some players were traded midseason and therefore appear more than once, let's merge them
    def merge_duplicates(self, df):
        duplicate = df.pivot_table(index=['PLAYER'], aggfunc='size')
        duplicate = duplicate[duplicate != 1]
        for player in duplicate.index:
            df1 = df.loc[df['PLAYER'] == player].fillna(0)   
            # create merged row
            merged = df1.iloc[0]
            total_games = df1['G'].sum()
            merged['G'] = total_games
            merged['GS'] = df1['GS'].sum()
            
            weighted_stats = pd.DataFrame()
            for index, row in df1._get_numeric_data().iterrows():
                weight = row['G']/total_games #calculate weight
                new_row = row*weight #muliply weight to get new values
                weighted_stats = weighted_stats.append(new_row) #add to df
            weighted_stats = weighted_stats.sum(axis=0, skipna=True)
            
            merged_stats = ['MP','FG','FGA','3P','3PA','2P','2PA','FT','FTA', 
                            'ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS']
            
            for stat in merged_stats:
                merged[stat] = weighted_stats[stat]
            
            # calulate % stats
            merged['FG%'] = merged['FG']/merged['FGA']
            merged['3P%'] = merged['3P']/merged['3PA']
            merged['2P%'] = merged['2P']/merged['2PA']
            merged['eFG%'] = (merged['FG']+0.5*merged['3P'])/merged['FGA']
            merged['FT%'] = merged['FT']/merged['FTA']
            merged=merged.fillna(0)
            
            df = df[df.PLAYER != player]
            df = df.append(merged)
            #print(df.loc[df['PLAYER']] == player)
            print(df.tail)
            #print(df.info())
        return df
        

In [187]:
df2 = pd.read_csv('2017.csv')
df2 = df2.fillna(0)
scraper1 = DataScraper()
scraper1.merge_duplicates(df2)

<ipython-input-186-62367b90215e>:53: RuntimeWarning: invalid value encountered in double_scalars
  merged['3P%'] = merged['3P']/merged['3PA']
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
<ipython-input-186-62367b90215e>:56: RuntimeWarning: invalid value encounter

<bound method NDFrame.tail of                PLAYER POS  AGE TEAM   G  GS         MP        FG        FGA  \
0       Kent Bazemore  SF   27  ATL  73  64  26.900000  4.000000   9.900000   
1     DeAndre' Bembry  SF   22  ATL  38   1   9.800000  1.200000   2.600000   
2     Lamar Patterson  PG   35  ATL  17   2  14.500000  1.400000   3.400000   
3     Malcolm Delaney  PG   27  ATL  73   2  17.100000  2.000000   5.300000   
4       Mike Dunleavy  SF   36  ATL  30   0  15.800000  1.900000   4.300000   
..                ...  ..  ...  ...  ..  ..        ...       ...        ...   
538  Tomáš Satoranský  SG   25  WAS  57   3  12.600000  1.100000   2.600000   
539       Jason Smith   C   30  WAS  74   3  14.400000  2.400000   4.400000   
540   Marcus Thornton  SG   29  WAS  33   1  17.400000  2.500000   6.400000   
541         John Wall  PG   26  WAS  78  78  36.400000  8.300000  18.400000   
94       Andrew Bogut   C   32  CLE  27  21  21.607407  1.444444   2.985185   

          FG%  ...   

/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:


<bound method NDFrame.tail of                PLAYER POS  AGE TEAM   G  GS         MP        FG        FGA  \
0       Kent Bazemore  SF   27  ATL  73  64  26.900000  4.000000   9.900000   
1     DeAndre' Bembry  SF   22  ATL  38   1   9.800000  1.200000   2.600000   
2     Lamar Patterson  PG   35  ATL  17   2  14.500000  1.400000   3.400000   
3     Malcolm Delaney  PG   27  ATL  73   2  17.100000  2.000000   5.300000   
4       Mike Dunleavy  SF   36  ATL  30   0  15.800000  1.900000   4.300000   
..                ...  ..  ...  ...  ..  ..        ...       ...        ...   
22   Bojan Bogdanović  SF   27  BRK  81  54  25.680247  4.643210  10.425926   
353  Brandon Jennings  PG   27  NYK  81  13  22.243210  2.374074   6.604938   
71      Briante Weber  PG   24  CHO  20   0  10.240000  1.220000   2.975000   
337       Buddy Hield  SG   24  NOP  82  55  23.052439  3.962195   9.367073   
87      Cameron Payne  PG   22  CHI  31   0  14.900000  2.093548   6.216129   

          FG%  ...  

/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

...

/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
<ipython-input-186-62367b90215e>:53: RuntimeWarning: invalid value encountered in double_scalars
  merged['3P%'] = merged['3P']/merged['3PA']
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/ser

<bound method NDFrame.tail of               PLAYER POS  AGE TEAM   G  GS         MP        FG        FGA  \
0      Kent Bazemore  SF   27  ATL  73  64  26.900000  4.000000   9.900000   
1    DeAndre' Bembry  SF   22  ATL  38   1   9.800000  1.200000   2.600000   
3    Malcolm Delaney  PG   27  ATL  73   2  17.100000  2.000000   5.300000   
5       Tim Hardaway  SG   24  ATL  79  30  27.300000  5.300000  11.500000   
6      Dwight Howard   C   31  ATL  74  74  29.700000  5.200000   8.300000   
..               ...  ..  ...  ...  ..  ..        ...       ...        ...   
204     Lou Williams  SG   30  HOU  81   1  24.625926  5.259259  12.359259   
155    Mason Plumlee   C   26  DEN  81  64  26.533333  4.166667   7.700000   
172      Matt Barnes  SF   36  GSW  74  18  24.002703  2.483784   6.397297   
4      Mike Dunleavy  SF   36  ATL  53   2  15.843396  1.769811   4.126415   
66     Miles Plumlee   C   28  CHO  45  12  10.768889  0.957778   2.013333   

          FG%  ...       ORB     

<bound method NDFrame.tail of               PLAYER POS  AGE TEAM    G  GS         MP        FG        FGA  \
0      Kent Bazemore  SF   27  ATL   73  64  26.900000  4.000000   9.900000   
1    DeAndre' Bembry  SF   22  ATL   38   1   9.800000  1.200000   2.600000   
3    Malcolm Delaney  PG   27  ATL   73   2  17.100000  2.000000   5.300000   
5       Tim Hardaway  SG   24  ATL   79  30  27.300000  5.300000  11.500000   
6      Dwight Howard   C   31  ATL   74  74  29.700000  5.200000   8.300000   
..               ...  ..  ...  ...  ...  ..        ...       ...        ...   
121       Quinn Cook  PG   23  DAL   45  19  14.848889  1.413333   2.922222   
61       Roy Hibbert   C   30  CHO   48  13  14.225000  1.700000   3.037500   
365   Semaj Christon  SG   23  OKC  132   7  15.354545  1.612121   4.272727   
393      Serge Ibaka  PF   27  ORL   79  79  30.645570  5.954430  12.454430   
60     Spencer Hawes  PF   28  CHO   54   1  14.768519  2.507407   5.109259   

          FG%  ...   

/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

0

In [52]:
if not os.path.exists(os.path.join(DATA_DIR,'PlayerStats')):
    os.mkdir(os.path.join(DATA_DIR,'PlayerStats'))

SEASONS = [2017,2018,2019,2020,2021]
TEAMS  = ['ATL','BRK','BOS','CHO','CHI','CLE','DAL','DEN','DET','GSW','HOU','IND','LAC','LAL','MEM',
          'MIA','MIL','MIN','NOP','NYK','OKC','ORL','PHI','PHO','POR','SAC','SAS','TOR','UTA','WAS']

scraper = DataScraper()
# Comment out season dates in SEASON_DATES in constants.py to extract data for specific seasons

for season in SEASONS:
    df = pd.DataFrame()
    for team in TEAMS:
        print(team)
        df1 = scraper.get_roster_stats(team, season)
        df1 = scraper.merge_duplicates(df)
        df = df.append(df1)
    df.to_csv(DATA_DIR+'/PlayerStats/'+f'{season-1}-{str(season)[2:]}+.csv', index=False)

ATL
BRK
BOS
CHO
CHI
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
MIA
MIL
MIN
NOP
NYK
OKC
ORL
PHI
PHO
POR
SAC
SAS
TOR
UTA
WAS
ATL
BRK
BOS
CHO
CHI
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
MIA
MIL
MIN
NOP
NYK
OKC
ORL
PHI
PHO
POR
SAC
SAS
TOR
UTA
WAS
ATL
BRK
BOS
CHO
CHI
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
MIA
MIL
MIN
NOP
NYK
OKC
ORL
PHI
PHO
POR
SAC
SAS
TOR
UTA
WAS
ATL
BRK
BOS
CHO
CHI
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
MIA
MIL
MIN
NOP
NYK
OKC
ORL
PHI
PHO
POR
SAC
SAS
TOR
UTA
WAS
ATL
BRK
BOS
CHO
CHI
CLE
DAL
DEN
DET
GSW
HOU
IND
LAC
LAL
MEM
MIA
MIL
MIN
NOP
NYK
OKC
ORL
PHI
PHO
POR
SAC
SAS
TOR
UTA
WAS
